In [1]:
import os

In [2]:
os.chdir('/content/drive/MyDrive/Molecule prediction')

In [3]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric
!pip install ogb

     |████████████████████████████████| 2.5MB 9.5MB/s 
     |████████████████████████████████| 1.5MB 6.6MB/s 
     |████████████████████████████████| 225kB 5.8MB/s 
     |████████████████████████████████| 235kB 36.6MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 71kB 4.5MB/s 
  Created wheel for littleutils: filename=littleutils-0.2.2-cp37-none-any.whl size=7050 sha256=a9764ee48560ce2ad3cf9e8c876f4f54261b83ab762a1b35c2310665a080c8ab
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built littleutils


In [4]:
!pip install -q git+https://github.com/snap-stanford/deepsnap.git

In [5]:
import torch

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Node Embbeding

In [7]:
import torch
import torch.nn.functional as F

print(torch.__version__)

from torch_geometric.nn import GATConv

import torch_geometric.transforms as T

1.8.1+cu101


In [8]:
class Node_Embbedding(torch.nn.Module):

  def __init__(self,input_dim,hidden_dim,output_dim,dropout):

    super(Node_Embbedding,self).__init__()

    self.conv1=GATConv(input_dim,hidden_dim)
    self.conv2=GATConv(hidden_dim,hidden_dim)
    self.conv3=GATConv(hidden_dim,hidden_dim)

    self.bn1=torch.nn.BatchNorm1d(hidden_dim)
    self.bn2=torch.nn.BatchNorm1d(hidden_dim)
    self.bn3=torch.nn.BatchNorm1d(hidden_dim)


    self.fc=torch.nn.Linear(hidden_dim,output_dim)

    self.dropout=dropout
    self.softmax = torch.nn.LogSoftmax()


  def reset_parameters(self):

    self.conv1.reset_parameters()
    self.conv2.reset_parameters()
    self.conv3.reset_parameters()

    self.bn1.reset_parameters()
    self.bn2.reset_parameters()
    self.bn3.reset_parameters()

  def forward(self,x,adj_t):

    x=self.conv1(x,adj_t)
    x=self.bn1(x)
    x=F.dropout(F.relu(x,True),p=self.dropout,training=self.training)

    x=self.conv2(x,adj_t)
    x=self.bn2(x)
    x=F.dropout(F.relu(x,True),p=self.dropout,training=self.training)

    x=self.conv3(x,adj_t)
    x=self.bn3(x)
    x=F.dropout(F.relu(x,True),p=self.dropout,training=self.training)

    return x



    

# Graph Classifier

In [9]:
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.data import DataLoader
from tqdm.notebook import tqdm

# Load the dataset 
dataset = PygGraphPropPredDataset(name='ogbg-molhiv')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

split_idx = dataset.get_idx_split()

# Check task type
print('Task type: {}'.format(dataset.task_type))

Device: cuda
Task type: binary classification


In [10]:
# Load the data sets into dataloader
# We will train the graph classification task on a batch of 32 graphs
# Shuffle the order of graphs for training set
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False, num_workers=0)

In [11]:
# Please do not change the args
args = {
    'device': device,
    'num_layers': 5,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.001,
    'epochs': 100,
}
args

{'device': 'cuda',
 'dropout': 0.5,
 'epochs': 100,
 'hidden_dim': 256,
 'lr': 0.001,
 'num_layers': 5}

In [22]:
from ogb.graphproppred.mol_encoder import AtomEncoder
from torch_geometric.nn import global_mean_pool

class GCN_Graph(torch.nn.Module):
    def __init__(self, hidden_dim, output_dim, dropout,pool=global_mean_pool):
        super(GCN_Graph, self).__init__()

        self.node_encoder = AtomEncoder(hidden_dim)

        self.gnn_node = Node_Embbedding(hidden_dim, hidden_dim,
            hidden_dim, dropout)

        self.pool = pool

        self.linear = torch.nn.Linear(hidden_dim, output_dim)


    def reset_parameters(self):
      self.gnn_node.reset_parameters()
      self.linear.reset_parameters()

    def forward(self, batched_data):
        
        x, edge_index, batch = batched_data.x, batched_data.edge_index, batched_data.batch
        embed = self.node_encoder(x)

        x = self.gnn_node(embed, edge_index)
        x = self.pool(x, batch)

        out=self.linear(x)

        return out

In [23]:
def train(model, device, data_loader, optimizer, loss_fn):

    model.train()
    loss = 0

    for step, batch in enumerate(tqdm(data_loader, desc="Iteration")):
      batch = batch.to(device)

      if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
          pass
      else:
        is_labeled = batch.y == batch.y

        optimizer.zero_grad()
        output = model(batch)[is_labeled]
        loss = loss_fn(output, batch.y.float()[is_labeled])

        loss.backward()
        optimizer.step()

    return loss.item()

In [24]:
def eval(model, device, loader, evaluator):
    model.eval()
    y_true = []
    y_pred = []

    for step, batch in enumerate(loader):
        batch = batch.to(device)

        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred = model(batch)

            y_true.append(batch.y.view(pred.shape).detach().cpu())
            y_pred.append(pred.detach().cpu())

    y_true = torch.cat(y_true, dim = 0).numpy()
    y_pred = torch.cat(y_pred, dim = 0).numpy()

    input_dict = {"y_true": y_true, "y_pred": y_pred}

    return evaluator.eval(input_dict)

# Mean Pooling

In [18]:
model = GCN_Graph(args['hidden_dim'],
            dataset.num_tasks,args['dropout']).to(device)
evaluator = Evaluator(name='ogbg-molhiv')

In [19]:
model.reset_parameters()

In [20]:
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

best_model = None
best_valid_acc = 0
paitient=10

for epoch in range(1, 1 + args["epochs"]):
  print()
  print('Training...')
  loss = train(model, device, train_loader, optimizer, loss_fn)

  print('Evaluating...')
  train_result = eval(model, device, train_loader, evaluator)
  val_result = eval(model, device, valid_loader, evaluator)
  test_result = eval(model, device, test_loader, evaluator)

  train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
      paitient=10

  else:
    paitient-=1

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')
  
  
  if paitient==0:
    break

Training...



Evaluating...
Epoch: 01, Loss: 0.0241, Train: 72.32%, Valid: 74.01% Test: 69.88%
Training...



Evaluating...
Epoch: 02, Loss: 0.0462, Train: 72.67%, Valid: 73.32% Test: 72.36%
Training...



Evaluating...
Epoch: 03, Loss: 0.6115, Train: 69.83%, Valid: 75.17% Test: 70.51%
Training...



Evaluating...
Epoch: 04, Loss: 0.7461, Train: 73.18%, Valid: 74.60% Test: 71.47%
Training...


...


Evaluating...
Epoch: 16, Loss: 0.0278, Train: 76.45%, Valid: 76.86% Test: 73.99%
Training...



Evaluating...
Epoch: 17, Loss: 0.3515, Train: 74.92%, Valid: 76.95% Test: 72.91%
Training...



Evaluating...
Epoch: 18, Loss: 0.0302, Train: 75.78%, Valid: 74.75% Test: 72.03%
Training...



Evaluating...
Epoch: 19, Loss: 0.0224, Train: 75.58%, Valid: 75.44% Test: 73.81%
Training...



Evaluating...
Epoch: 20, Loss: 0.0514, Train: 76.18%, Valid: 77.15% Test: 74.23%


In [21]:
train_acc = eval(best_model, device, train_loader, evaluator)[dataset.eval_metric]
valid_acc = eval(best_model, device, valid_loader, evaluator)[dataset.eval_metric]
test_acc = eval(best_model, device, test_loader, evaluator)[dataset.eval_metric]

print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

Best model: Train: 74.58%, Valid: 78.43% Test: 71.50%


# ADD Polling

In [26]:
from torch_geometric.nn import global_add_pool

In [29]:
model = GCN_Graph(args['hidden_dim'],
            dataset.num_tasks,args['dropout'],global_add_pool).to(device)
evaluator = Evaluator(name='ogbg-molhiv')

In [30]:
model.reset_parameters()

In [31]:
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

best_model = None
best_valid_acc = 0
paitient=10

for epoch in range(1, 1 + args["epochs"]):
  print()
  print('Training...')
  loss = train(model, device, train_loader, optimizer, loss_fn)

  print('Evaluating...')
  train_result = eval(model, device, train_loader, evaluator)
  val_result = eval(model, device, valid_loader, evaluator)
  test_result = eval(model, device, test_loader, evaluator)

  train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
      paitient=10

  else:
    paitient-=1

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')
  
  
  if paitient==0:
    break


Training...



Evaluating...
Epoch: 01, Loss: 0.0381, Train: 62.35%, Valid: 61.70% Test: 60.41%

Training...



Evaluating...
Epoch: 02, Loss: 0.0322, Train: 68.83%, Valid: 62.74% Test: 65.02%

Training...



Evaluating...
Epoch: 03, Loss: 0.0826, Train: 72.19%, Valid: 69.71% Test: 65.84%

Training...



Evaluating...
Epoch: 04, Loss: 0.0157, Train: 72.08%, Valid: 70.71% Test: 63.93%

Training...


...


Evaluating...
Epoch: 25, Loss: 0.0150, Train: 80.90%, Valid: 73.85% Test: 75.40%

Training...



Evaluating...
Epoch: 26, Loss: 0.0316, Train: 81.75%, Valid: 76.85% Test: 75.34%

Training...



Evaluating...
Epoch: 27, Loss: 0.0272, Train: 81.85%, Valid: 78.09% Test: 77.22%

Training...



Evaluating...
Epoch: 28, Loss: 0.0383, Train: 80.30%, Valid: 77.73% Test: 72.31%

Training...



Evaluating...
Epoch: 29, Loss: 0.0232, Train: 81.07%, Valid: 78.99% Test: 76.77%


In [32]:
train_acc = eval(best_model, device, train_loader, evaluator)[dataset.eval_metric]
valid_acc = eval(best_model, device, valid_loader, evaluator)[dataset.eval_metric]
test_acc = eval(best_model, device, test_loader, evaluator)[dataset.eval_metric]

print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

Best model: Train: 80.47%, Valid: 82.03% Test: 76.72%


# Max poolin

In [38]:
from torch_geometric.nn import global_max_pool

In [39]:
model = GCN_Graph(args['hidden_dim'],
            dataset.num_tasks,args['dropout'],global_max_pool).to(device)
evaluator = Evaluator(name='ogbg-molhiv')

In [40]:
model.reset_parameters()

In [41]:
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

best_model = None
best_valid_acc = 0
paitient=10

for epoch in range(1, 1 + args["epochs"]):
  print()
  print('Training...')
  loss = train(model, device, train_loader, optimizer, loss_fn)

  print('Evaluating...')
  train_result = eval(model, device, train_loader, evaluator)
  val_result = eval(model, device, valid_loader, evaluator)
  test_result = eval(model, device, test_loader, evaluator)

  train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
      paitient=10

  else:
    paitient-=1

  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')
  
  
  if paitient==0:
    break


Training...



Evaluating...
Epoch: 01, Loss: 0.8339, Train: 64.26%, Valid: 67.19% Test: 65.37%

Training...



Evaluating...
Epoch: 02, Loss: 0.6806, Train: 63.60%, Valid: 65.25% Test: 58.44%

Training...



Evaluating...
Epoch: 03, Loss: 0.0443, Train: 72.36%, Valid: 69.62% Test: 68.50%

Training...



Evaluating...
Epoch: 04, Loss: 0.0257, Train: 67.07%, Valid: 69.00% Test: 53.55%

Training...


...


Evaluating...
Epoch: 12, Loss: 0.0274, Train: 72.43%, Valid: 66.52% Test: 69.11%

Training...



Evaluating...
Epoch: 13, Loss: 0.1727, Train: 73.53%, Valid: 63.88% Test: 65.14%

Training...



Evaluating...
Epoch: 14, Loss: 0.6478, Train: 73.26%, Valid: 65.94% Test: 66.06%

Training...



Evaluating...
Epoch: 15, Loss: 0.0387, Train: 72.74%, Valid: 69.50% Test: 65.84%

Training...



Evaluating...
Epoch: 16, Loss: 0.2168, Train: 73.47%, Valid: 68.87% Test: 67.17%


In [42]:
train_acc = eval(best_model, device, train_loader, evaluator)[dataset.eval_metric]
valid_acc = eval(best_model, device, valid_loader, evaluator)[dataset.eval_metric]
test_acc = eval(best_model, device, test_loader, evaluator)[dataset.eval_metric]

print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

Best model: Train: 74.57%, Valid: 71.69% Test: 65.93%
